In [ ]:
import sys


if sys.version.split()[0] != '3.10.12':
    print(sys.version)
    raise RuntimeError('Wrong Python version. Go to ENVIRONMENT settings, Stop machine, Start machine')


import tensorflow
if tensorflow.__version__ != '2.13.0':
    raise RuntimeError('Wrong TF version. Go to ENVIRONMENT settings, Stop machine, Start machine')


print('\nPython version: OK')
print('TensorFlow version: OK')

2023-12-22 14:25:39.382590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 14:25:39.458153: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 14:25:39.458943: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-22 14:25:41.151728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Python version: OK
TensorFlow version: OK


In [ ]:
import tensorflow as tf
import os

In [ ]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.032,
    'frame_step_in_s': 0.01,
    'num_mel_bins': 20,
    'lower_frequency': 20,
    'upper_frequency': 6000,
}

TRAINING_ARGS = {
    'batch_size': 25,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 80 # 10
}
final_sparsity = 0.85

In [ ]:
train_ds = tf.data.Dataset.list_files('/tmp/yn-train/*')
test_ds = tf.data.Dataset.list_files('/tmp/yn-test/*')

In [ ]:
#import tensorflow as tf

num_files = len(train_ds)
shuffled_train_ds = train_ds.shuffle(num_files, reshuffle_each_iteration=False)

#train_ratio = 0.85
#num_train = int(train_ratio * num_files)

#train_ds = train_dataset.take(num_train)
#val_ds = train_dataset.skip(num_train)

In [ ]:
print(f'Train set size {len(train_ds)}')
print(f'Test set size {len(test_ds)}')

Train set size 1600
Test set size 200


In [ ]:
for x in train_ds.take(5):
    print(x)

tf.Tensor(b'/tmp/yn-train/yes_106a6183_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'/tmp/yn-train/yes_3b8406c0_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'/tmp/yn-train/yes_ced4e2a1_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'/tmp/yn-train/yes_73124b26_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'/tmp/yn-train/no_d94eb94f_nohash_0.wav', shape=(), dtype=string)


In [ ]:
import preprocessing

In [ ]:
from preprocessing import LABELS
from preprocessing import AudioReader
from preprocessing import MelSpectrogram

audio_reader = AudioReader(tf.int16, 16000)
mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id

batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = (train_ds
                .map(audio_reader.get_audio_and_label)
                .map(mel_spec_processor.get_mel_spec_and_label)
                .map(prepare_for_training)
                .batch(batch_size)
                .cache()
            )
#val_ds = (val_ds
#                .map(audio_reader.get_audio_and_label)
#                .map(mel_spec_processor.get_mel_spec_and_label)
#                .map(prepare_for_training)
#                .batch(batch_size)
#            )
test_ds = (test_ds
                .map(audio_reader.get_audio_and_label)
                .map(mel_spec_processor.get_mel_spec_and_label)
                .map(prepare_for_training)
                .batch(batch_size)
            )


2023-12-22 14:26:22.115463: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 FMA
2023-12-22 14:26:22.118354: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [ ]:
for example_batch, example_labels in train_ds.take(1):
    print(example_batch.shape),
    print(example_labels)

(25, 97, 20, 1)
tf.Tensor([0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 0], shape=(25,), dtype=int64)
2023-12-22 14:26:30.040219: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Conv2D(filters=16, kernel_size=[3, 3], stride=[2, 2], use_bias=False, padding=’valid’)
BatchNormalization()
ReLU()
Conv2D(filters=16, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding = 'same')
BatchNormalization()
ReLU()
Conv2D(filters=16, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding = 'same')
BatchNormalization()
ReLU()
Conv2D(filters=16, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding = 'same')
BatchNormalization()
ReLU()
GlobalAveragePooling2D()
Dense(len(LABELS))
Softmax()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(LABELS)),
    tf.keras.layers.Softmax(), 
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 9, 16)         144       
                                                                 
 batch_normalization (Batch  (None, 48, 9, 16)         64        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 48, 9, 16)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 9, 16)         2304      
                                                                 
 batch_normalization_1 (Bat  (None, 48, 9, 16)         64        
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 48, 9, 16)         0

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.2,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [ ]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 48, 9, 16)         290       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_batch_  (None, 48, 9, 16)         65        
 normalization (PruneLowMag                                      
 nitude)                                                         
                                                                 
 prune_low_magnitude_re_lu   (None, 48, 9, 16)         1         
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_conv2d  (None, 48, 9, 16)         4610      
 _1 (PruneLowMagnitude)                                          
                                                        

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

lr_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate,
    len(train_ds) * epochs,
    end_learning_rate,
)
optimizer = tf.optimizers.Adam(learning_rate=lr_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks)


Epoch 1/80
64/64 [==============================] - 10s 81ms/step - loss: 0.0064 - sparse_categorical_accuracy: 0.9981
Epoch 2/80
64/64 [==============================] - 5s 81ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9912
Epoch 3/80
64/64 [==============================] - 5s 79ms/step - loss: 0.0248 - sparse_categorical_accuracy: 0.9887
Epoch 4/80
64/64 [==============================] - 5s 79ms/step - loss: 0.0172 - sparse_categorical_accuracy: 0.9962
Epoch 5/80
64/64 [==============================] - 5s 79ms/step - loss: 0.0080 - sparse_categorical_accuracy: 0.9962
Epoch 6/80
64/64 [==============================] - 5s 79ms/step - loss: 0.0053 - sparse_categorical_accuracy: 0.9981
Epoch 7/80
64/64 [==============================] - 5s 80ms/step - loss: 0.0047 - sparse_categorical_accuracy: 0.9987
Epoch 8/80
64/64 [==============================] - 5s 79ms/step - loss: 0.0043 - sparse_categorical_accuracy: 0.9987
Epoch 9/80
64/64 [==============================] - 5s 

In [ ]:
import numpy as np

for layer in model_for_pruning.layers:
    if isinstance(layer, tf.keras.layers.Wrapper):
        weights = layer.trainable_weights
    else:
        weights = layer.weights
    for weight in weights:
        weight_size = weight.numpy().size
        zero_num = np.count_nonzero(weight == 0)
        print(f'{weight.name}: {zero_num/weight_size:.2%} sparsity',
              f'({zero_num}/{weight_size})'
        )

conv2d/kernel:0: 84.72% sparsity (122/144)
batch_normalization/gamma:0: 0.00% sparsity (0/16)
batch_normalization/beta:0: 0.00% sparsity (0/16)
conv2d_1/kernel:0: 84.98% sparsity (1958/2304)
batch_normalization_1/gamma:0: 0.00% sparsity (0/16)
batch_normalization_1/beta:0: 0.00% sparsity (0/16)
conv2d_2/kernel:0: 84.98% sparsity (1958/2304)
batch_normalization_2/gamma:0: 0.00% sparsity (0/16)
batch_normalization_2/beta:0: 0.00% sparsity (0/16)
conv2d_3/kernel:0: 84.98% sparsity (1958/2304)
batch_normalization_3/gamma:0: 0.00% sparsity (0/16)
batch_normalization_3/beta:0: 0.00% sparsity (0/16)
dense/kernel:0: 84.38% sparsity (27/32)
dense/bias:0: 0.00% sparsity (0/2)


In [ ]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)
print(test_loss, test_accuracy)

8/8 [==============================] - 1s 97ms/step - loss: 0.0266 - sparse_categorical_accuracy: 0.9900
0.02657555602490902 0.9900000095367432


In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
#val_loss = history.history['val_loss'][-1]
#val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
#print(f'Validation Loss: {val_loss:.4f}')
#print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
#print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.0014
Training Accuracy: 99.94%

Test Loss: 0.0266
Test Accuracy: 99.00%


In [ ]:
import os
import time

timestamp = int(time.time())

save_model_dir = f'./saved_models/{timestamp}'
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

if not os.path.exists(save_model_dir):
    os.makedirs(save_model_dir)

model_for_export.save(save_model_dir)
tf_size = os.path.getsize(save_model_dir) / 1024
print(f'Original tf size (pruned model): {tf_size:.3f} KB')

INFO:tensorflow:Assets written to: ./saved_models_lorenzo/1703257074/assets
INFO:tensorflow:Assets written to: ./saved_models_lorenzo/1703257074/assets
Original tf size (pruned model): 0.000 KB


In [ ]:
!ls saved_models

In [ ]:
MODEL_NAME = 'model13'

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
tflite_model = converter.convert()


In [ ]:
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [ ]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

In [ ]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

In [ ]:
!ls tflite_models

In [ ]:
import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

In [ ]:
tflite_size = os.path.getsize(tflite_model_name) / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size (pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b0b0df8-8392-4edb-9f9f-6f8029fb1a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>